In [1]:
import os
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
import json


import time

import os
from dotenv import load_dotenv

load_dotenv()

# Supress warnings
import warnings
warnings.filterwarnings("ignore")

def load_rag_pipeline(name = "faiss_index"):
    # Load the index
    db = FAISS.load_local(name, OpenAIEmbeddings())
    # Expose this index in a retriever interface
    retriever = db.as_retriever()
    # Create a chain to answer questions
    return RetrievalQA.from_chain_type(
        llm=OpenAI(),
        chain_type="map_reduce",
        retriever=retriever,
        return_source_documents=True,
        verbose=True,
    )


def answer_question(question,qa):
    answer = qa(question)
    return answer

In [3]:
print("Building RAG pipeline...")
start = time.time()
# Load the RAG pipeline
qa = load_rag_pipeline()
print("Building took {0}s".format(time.time()-start))
print("Done building RAG pipeline")
print("Answering question...")
start = time.time()
# Answer a question
ans = answer_question("What is the first aid for a heart attack?",qa)


Building RAG pipeline...
Building took 0.0711214542388916s
Done building RAG pipeline
Answering question...


> Entering new RetrievalQA chain...

> Finished chain.


In [7]:
ans["result"]

' The first aid for a heart attack is to assess the situation first, check for responsiveness and pulse, ensure clear airway, and provide CPR if necessary.'

In [12]:
ans['source_documents'][0].metadata['source']

{'source': 'data/first aid.pdf', 'page': 5}

In [ ]:
import fitz

file_path = ans['source_documents'][0].metadata['source']
page_no = ans['source_documents'][0].metadata['page']

def extract_photo(file_path,page_no):
    pdf_file = fitz.open(file_path)
    # iterate over PDF pages 
    page = pdf_file[page_no]
  
    
        for image_index, img in enumerate(page.get_images(), start=1): 
    
            # get the XREF of the image 
            xref = img[0] 
    
            # extract the image bytes 
            base_image = pdf_file.extract_image(xref) 
            image_bytes = base_image["image"] 
    
            # get the image extension 
            image_ext = base_image["ext"] 
            
            file_name = os.path.splitext(os.path.basename(file_path))[0]

            name = os.path.splitext(os.path.basename(file_name))[0]
            # load it to PIL
            image = Image.open(io.BytesIO(image_bytes))
            image_path = os.path.join("./images", "image_"+ name +".png")
            image.save(image_path, image_ext)
            image_load_path = os.path.join("./images", "image_"+ name +".png")
            return image_load_path
    return None


In [16]:
import os
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS

import time

import os
from dotenv import load_dotenv

load_dotenv()

# Supress warnings
import warnings
warnings.filterwarnings("ignore")

def load_rag_pipeline(name = "faiss_index"):
    # Load the index
    db = FAISS.load_local(name, OpenAIEmbeddings())
    # Expose this index in a retriever interface
    retriever = db.as_retriever()
    # Create a chain to answer questions
    return RetrievalQA.from_chain_type(
        llm=OpenAI(),
        chain_type="map_reduce",
        retriever=retriever,
        return_source_documents=True,
        verbose=True,
    )


def answer_question(question,qa):
    context = "You are a helpful doctor that would provide patients with first aid information. Give them detailed and simple instructions. The input by the patient given seperated by hyphens ---{0}---"
    answer = qa(context.format(question))
    return answer

def report_to_doctor(answer):
    from openai import OpenAI

    client = OpenAI()

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant and you are required to send doctors a brief about the patients before they arrive."},
            {"role": "user", "content": "The details are are as follows {0}".format(answer)},
            {"role": "user", "content": "Can you give me a brief and useful description to be sent to the doctor. "}
        ]
    )
    return response

print("Building RAG pipeline...")
start = time.time()
# Load the RAG pipeline
qa = load_rag_pipeline()
print("Building took {0}s".format(time.time()-start))
print("Done building RAG pipeline")
print("Answering question...")
start = time.time()
# Answer a question
ans = answer_question("He has a severe head injury",qa)

print(ans['result'])

doc_res = report_to_doctor(ans['result'])


        



Building RAG pipeline...
Building took 0.07072687149047852s
Done building RAG pipeline
Answering question...


> Entering new RetrievalQA chain...


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-iaY77***************************************BYY1. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [15]:
print(doc_res)



NameError: name 'doc_res' is not defined